In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
iris_path = "/content/drive/MyDrive/ML/Lab 7/iris.csv"
df = pd.read_csv(iris_path)

In [ ]:
def discretize(df):
  for i in list(df):
    if i != "variety":
      col = df[i]
      print(i)
      bins = pd.cut(col, 3, labels = ["1", "2", "3"]) # Discretizes to bins
      df[i] = bins

In [ ]:
discretize(df)

sepal.length
sepal.width
petal.length
petal.width


In [ ]:
y = df['variety']
X = df.drop(['variety'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
def initialize_prior(y_train):
  py = []
  for i, j in enumerate(y_train.value_counts()):
    py.append(j / len(y_train))
  return py

In [ ]:
def generate_likelyhood_table(X_train, y_train):
  table = []
  for i in list(X_train):
    col = X_train[i]
    arr2 = []
    for j in list(y_train.unique()):
      rows = y_train.loc[y_train == j]
      arr = []
      for k in ["1", "2", "3"]:
        crows = rows.loc[col == k]
        arr.append(len(crows)/len(rows))
      arr2.append(arr)
    for k in ["1", "2", "3"]:
      arr = []
      for m in col.value_counts():
          arr.append(m / len(y_train))
    arr2.append(arr)
    table.append(arr2)
  return table

In [ ]:
def fit(X_train, y_train):
  py = initialize_prior(y_train)
  table = generate_likelyhood_table(X_train, y_train)
  return py, table
def predict(py, table, X_test, classes):

  preds = []
  for i, i1 in X_test.iterrows(): # enumerate through every instance
    prob_list = []
    for j, j1 in enumerate(y_train.unique()): # enumerate through every class
      prob = 1
      for k, k1 in enumerate(list(i1)): #enumerate thorugh every attribute
        prob *= table[k][j][int(k1) - 1]
        prob /= table[k][-1][int(k1) - 1]
      prob *= py[j]
      prob_list.append(prob)
    pred = np.argmax(np.array(prob_list))
    preds.append(classes[pred])
  return preds


In [ ]:
py, table = fit(X_train, y_train)
preds = predict(py, table, X_test, classes = list(y_train.unique()))

In [ ]:
gts = list(y_test)

In [ ]:
corr, tot = 0, len(y_test)
for i, j in zip(gts, preds):
  if i == j:
    corr += 1
print("Testing Accuracy:", corr / tot)
print("\nConfusion Matrix:\n", pd.crosstab(pd.Series(gts, name = "Actual"), pd.Series(preds, name = "Predicted")))

Testing Accuracy: 0.9666666666666667

Confusion Matrix:
 Predicted   Setosa  Versicolor  Virginica
Actual                                   
Setosa          11           0          0
Versicolor       0          13          0
Virginica        0           1          5
